In [1]:
# from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]

# model = SentenceTransformer('sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-lng-aligned')
# embeddings = model.encode(sentences)
# print(embeddings)

In [2]:
import pandas as pd

raw = pd.read_csv('cordlife_faqs1.csv')

lookup = {}
for a, b, *_ in raw.values:
    lookup[a] = b

raw

,prompt,response,Unnamed: 2
0,What is cord blood?,Cord blood is blood that remains in the umbili...,NaN
1,What are cord blood stem cells?,Cord blood stem cells are also known as Haemat...,NaN
2,Why should I save my babys cord blood stem cells?,There are several advantages of storing your b...,NaN
3,What can cord blood stem cells do?,Cord blood stem cells can:\r\n\r\nReplace and ...,NaN
4,How does a cord blood stem cell transplant work?,The purpose of a stem cell treatment is to rec...,NaN
...,...,...,...
58,When will the new GST change from 7% to 8% tak...,The new GST change from 7% to 8% will take eff...,NaN
59,How will the GST change affect my payments to ...,For Existing Clients\r\nThe revision in GST ra...,NaN
60,"With the absorption of the 1% GST increase, wh...",Any enrolment on or before 31 December 2022 wi...,NaN
61,What should I do if I am currently on an annua...,You can consider upgrading your current annual...,NaN


In [3]:
from sentence_transformers import SentenceTransformer, util

sentences = list(raw['prompt'].values)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
sentences_embeddings = [model.encode(s, convert_to_tensor=True) for s in sentences]


c:\Users\amir.rahman\AppData\Local\anaconda3\envs\cordlife\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def search(q, THRESHOLD=0.65):
    q_embedding = model.encode(q, convert_to_tensor=True)
    out = []
    for sentence, sentence_embedding in zip(sentences, sentences_embeddings):
        score = util.pytorch_cos_sim(q_embedding, sentence_embedding).item()
        out.append((sentence, score))
    out.sort(key=lambda x:-x[-1])
    if out[0][1] < THRESHOLD:
        return ['I don\'t know']
    top = out[:3]
    return [(qns, lookup[qns], score) for qns,score in top]

search('what is cordlife preferential plan and how would it benefit a parent like me')

[('How would Preferential Plan benefit a Cordlife Parent like me?',
  "As a gesture of our appreciation to our Cordlife parents, we've introduced the Preferential Plan which allows Cordlife Preferential Plan lets you enjoy: Protection against increases in Annual Instalment Fees. Protection against hikes in Goods and Services Tax (GST). The convenience of a one-time payment for the remaining balance of annual instalment fee. Multiple payment modes such as Child Development Account (CDA), credit card interest-free instalment plans & etc.                                                           Contact us to find out more about the full benefits.",
  0.9067820310592651),
 ('What is Cordlife Preferential Plan?',
  "As a gesture of our appreciation to our Cordlife parents, we've introduced the Preferential Plan which allows parents to prepay the balance of their annual fees. Multiple payment modes are available; including the use of your child's Child Development Account (CDA) for this one

In [5]:
search('what is CDA and how does it work')

[('What is a Child Development Account (CDA)?',
  'The CDA* is a special savings account for children to help build up the savings that can be spent on approved uses.\r\n\r\nYour child�s CDA can be opened at any OCBC Bank, DBS Bank or UOB Bank branch.\r\n\r\nYou can deposit cash into the CDA any time until 31 December of the year your child turns 12 years of age.\r\n\r\nFor more information on CDA, please visit www.babybonus.msf.gov.sg or ask our friendly consultants today!\r\n\r\n*A child is eligible to apply for a CDA if he/she is:\r\n\r\nBorn on or after 17 August 2008\r\nA Singaporean citizen (or becomes a citizen before the age of 12)\r\nThe parents are legally married',
  0.7746552228927612),
 ('How does the Child Development Account (CDA) work?',
  'The CDA has two components: the CDA First Step Grant and the Government Dollar-for-Dollar Matching.\r\n\r\nUnder the CDA First Step Grant, parents will receive an initial amount of $3,000 (from the Government�s existing contribution 

In [6]:
search('does cordlife plan to launch anything in the near future')

["I don't know"]

In [7]:
search('does cordlife have branches in europe or america')

["I don't know"]

In [8]:
search('how is cord blood stored?')

[("How is my baby's cord blood stored?",
  "Your baby's umbilical cord blood is stored within a US FDA-approved cryogenic storage pouch made of a special material that is specifically designed to withstand cryogenic temperatures. The pouch has two main segments (20% and 80%) that are attached integrally, and two test segments that are also integrally attached. The integral segments allow for additional testing on the associated unit to ensure that no samples are mixed up and that the cord blood remains viable. These tests are typically performed prior to a transplant. Having dual integrated segments addresses the possibility of future stem cell expansion. This means that when stem cell expansion is available commercially, you will be able to withdraw a portion of the stem cells for expansion while keeping the remainder in storage.",
  0.8244754672050476),
 ('What is cord blood?',
  'Cord blood is blood that remains in the umbilical cord following the birth of a baby and after the umbil